In [2]:
import simpy

+ Simulationsprozesse durch ``process`` realisiert
+ diese Prozesse werden in einem `environment` angesiedelt, über die sie interagieren

**Szenario**: Ein Drucker, der sich zu festen Zeiten in zwei möglichen Zuständen befindet:
+ Drucker arbeitet für die Dauer `Druckdauer`
+ Drucker wartet auf Druckauftrag `Wartezeit` 

In [4]:
def drucker(env):
    while True:
        print(f'Starte Druckvorgang {env.now}')
        druckdauer = 2
        yield env.timeout(druckdauer)

        print(f'Wartezeit auf Druckauftrag beginnt zur Zeit {env.now}')
        wartezeit = 10
        yield env.timeout(wartezeit)

Prozess `drucker` benötigt eine Referenz auf die Umgebung `env`, um neue Ereignisse zu starten.
+ Prozess ist als Endlosschleife angelegt
+ Sobald `yield` erreicht wird, gibt Prozess `drucker` die Kontrolle an das Environment (Simulation) ab.
+ Durch `env.timeout` Aufruf mit Übergabe des entspechenden Ereignisses (Hier `druckdauer` oder `wartezeit`) wird der Simulation signalisiert, auf dieses Ereignis zu warten.
+ Ausgabe des aktuellen Zustands aktuellen Simulationszeit

In [5]:
# erzeuge Simulationsumgebung
env = simpy.Environment()
env.process(drucker(env))

<Process(drucker) object at 0x7fb5f25974c0>

In [6]:
# starte Simulation
env.run(until=60)

Starte Druckvorgang 0
Wartezeit auf Druckauftrag beginnt zur Zeit 2
Starte Druckvorgang 12
Wartezeit auf Druckauftrag beginnt zur Zeit 14
Starte Druckvorgang 24
Wartezeit auf Druckauftrag beginnt zur Zeit 26
Starte Druckvorgang 36
Wartezeit auf Druckauftrag beginnt zur Zeit 38
Starte Druckvorgang 48
Wartezeit auf Druckauftrag beginnt zur Zeit 50


**Szenario**: Betrachte Druckerpool mit 2 Druckern. Jede Minute kommt ein neuer Druckauftrag an, dessen Ausführung jeweils 3 Minuten dauert. Berechne die Wartezeiten von Auftragsankunft bis Druckbeginn.



In [15]:
def druckerpool(env, auftrag, drucker, ankunft, druckdauer):
    # simuliere ankommende Druckaufträge
    yield env.timeout(ankunft)
    print(f"{auftrag} kommt an ztur Zeit {env.now}") # "Auftrag xx kommt an zur Zeit yy"
    t_ankunft = env.now

    # fordere Drucker an
    with drucker.request() as req:
        yield req
        # Auftrag wird gedruckt sobald Ressource frei
        wartezeit = env.now - t_ankunft
        print(f"{auftrag} Druckbeginn zur Zeit {env.now}, Wartezeit: {wartezeit}") # "Auftrag xx Druckbeginn zur Zeit yy"
        yield env.timeout(druckdauer)
        print(f"{auftrag} Druckende zur Zeit {env.now}") # "Auftrag xx Druckende zur Zeit yy"

+ mit `request()` wird Drucker zur Bearbeitung angefordert bzw. u.U. auf einen freien Drucker gewartet
+ wird `request` zusammen mt `with` angewendet, dann wird die angeforderte Ressource nach Bearbeitung eines Auftrags automatisch wieder freigegeben
+ Ressource wird mit FCFS-Disziplin beschäftigt
+ Ressourcen brauchen einen Verweis auf Simulationsumgebung und eine Kapazitätsangabe (später)

In [16]:
# Erzeuge Simulationsumgebung
env = simpy.Environment()
drucker = simpy.Resource(env, capacity=2)

In [17]:
# betrachte die ersten 10 Druckaufträge
for i in range(1, 11):
    env.process(druckerpool(env, f"Auftrag {i}", drucker, i, 3))

In [18]:
# starte simulation
env.run()

Auftrag 1 kommt an ztur Zeit 1
Auftrag 1 Druckbeginn zur Zeit 1, Wartezeit: 0
Auftrag 2 kommt an ztur Zeit 2
Auftrag 2 Druckbeginn zur Zeit 2, Wartezeit: 0
Auftrag 3 kommt an ztur Zeit 3
Auftrag 4 kommt an ztur Zeit 4
Auftrag 1 Druckende zur Zeit 4
Auftrag 3 Druckbeginn zur Zeit 4, Wartezeit: 1
Auftrag 5 kommt an ztur Zeit 5
Auftrag 2 Druckende zur Zeit 5
Auftrag 4 Druckbeginn zur Zeit 5, Wartezeit: 1
Auftrag 6 kommt an ztur Zeit 6
Auftrag 7 kommt an ztur Zeit 7
Auftrag 3 Druckende zur Zeit 7
Auftrag 5 Druckbeginn zur Zeit 7, Wartezeit: 2
Auftrag 8 kommt an ztur Zeit 8
Auftrag 4 Druckende zur Zeit 8
Auftrag 6 Druckbeginn zur Zeit 8, Wartezeit: 2
Auftrag 9 kommt an ztur Zeit 9
Auftrag 10 kommt an ztur Zeit 10
Auftrag 5 Druckende zur Zeit 10
Auftrag 7 Druckbeginn zur Zeit 10, Wartezeit: 3
Auftrag 6 Druckende zur Zeit 11
Auftrag 8 Druckbeginn zur Zeit 11, Wartezeit: 3
Auftrag 7 Druckende zur Zeit 13
Auftrag 9 Druckbeginn zur Zeit 13, Wartezeit: 4
Auftrag 8 Druckende zur Zeit 14
Auftrag 10

**Szenario** Druckerpool mit zwei Druckern. Ankunftsdauer liegt zwischen 1 und 4 Minuten. Druckdauer ist eine ganzzahlige Zahl zwischen 2 und 6 Minuten

In [33]:
import random

In [38]:
def druckerpool(env, auftrag, drucker, ankunft, druckdauer):
    # simuliere ankommende Druckaufträge
    yield env.timeout(ankunft)
    print(f"{auftrag} kommt an zur Zeit {env.now}") # "Auftrag xx kommt an zur Zeit yy"
    t_ankunft = env.now

    # fordere Drucker an
    with drucker.request() as req:
        yield req
        # Auftrag wird gedruckt sobald Ressource frei
        wartezeit = env.now - t_ankunft
        print(f"{auftrag} Druckbeginn zur Zeit {env.now} (Wartezeit: {wartezeit} und Dauer {druckdauer})") # "Auftrag xx Druckbeginn zur Zeit yy"
        yield env.timeout(druckdauer)
        print(f"{auftrag} Druckende zur Zeit {env.now}") # "Auftrag xx Druckende zur Zeit yy"

In [39]:
env = simpy.Environment()
drucker = simpy.Resource(env, capacity=2)

In [40]:
# betrachte die ersten 10 Druckaufträge
t_0 = 0
for i in range(1, 11):
    t_0 += random.randint(1, 4)
    env.process(druckerpool(env, f"Auftrag {i}", drucker, t_0, random.randint(2, 6)))

In [41]:
env.run()

Auftrag 1 kommt an zur Zeit 3
Auftrag 1 Druckbeginn zur Zeit 3 (Wartezeit: 0 und Dauer 2)
Auftrag 2 kommt an zur Zeit 4
Auftrag 2 Druckbeginn zur Zeit 4 (Wartezeit: 0 und Dauer 4)
Auftrag 1 Druckende zur Zeit 5
Auftrag 3 kommt an zur Zeit 6
Auftrag 3 Druckbeginn zur Zeit 6 (Wartezeit: 0 und Dauer 3)
Auftrag 4 kommt an zur Zeit 7
Auftrag 5 kommt an zur Zeit 8
Auftrag 2 Druckende zur Zeit 8
Auftrag 4 Druckbeginn zur Zeit 8 (Wartezeit: 1 und Dauer 6)
Auftrag 3 Druckende zur Zeit 9
Auftrag 5 Druckbeginn zur Zeit 9 (Wartezeit: 1 und Dauer 2)
Auftrag 5 Druckende zur Zeit 11
Auftrag 6 kommt an zur Zeit 12
Auftrag 6 Druckbeginn zur Zeit 12 (Wartezeit: 0 und Dauer 2)
Auftrag 7 kommt an zur Zeit 14
Auftrag 4 Druckende zur Zeit 14
Auftrag 6 Druckende zur Zeit 14
Auftrag 7 Druckbeginn zur Zeit 14 (Wartezeit: 0 und Dauer 4)
Auftrag 8 kommt an zur Zeit 18
Auftrag 7 Druckende zur Zeit 18
Auftrag 8 Druckbeginn zur Zeit 18 (Wartezeit: 0 und Dauer 3)
Auftrag 9 kommt an zur Zeit 20
Auftrag 9 Druckbeginn 

# M/M/2 - Supermarktkasse

In [42]:
# mittlerer Ankunftsabstand a, mittlere Bediendauer b [sekunden]
a = 25
b = 45

In [43]:
def kassensystem(env, kundin, kasse, ankunft, kassierdauer):
    # Simuliere ankommende Kundinnen
    yield env.timeout(ankunft)
    print(f"{env.now} {kundin} Ankunft in Warteschlange")

    t_ankunft = env.now

    #fordere Kassiervorgang an
    with kasse.request() as req:
        yield req
        wartezeit = env.now - t_ankunft
        print(f"{env.now} {kundin} Kassiervorgang beginnt (Wartezeit {wartezeit})")
        
        yield env.timeout(kassierdauer)
        print(f"{env.now} {kundin} Kassiervorgang endet")

In [44]:
env = simpy.Environment()
kasse = simpy.Resource(env, capacity = 2)

In [45]:
# betrachte die ersten 10 Kundinnen
t_0 = 0
for i in range(1, 11):
    t_0 += random.expovariate(1/a)
    env.process(kassensystem(env, f"Kundin {i}", kasse, t_0, random.expovariate(1/b)))

In [46]:
env.run()

22.957681901420425 Kundin 1 Ankunft in Warteschlange
22.957681901420425 Kundin 1 Kassiervorgang beginnt (Wartezeit 0.0)
33.78762702788011 Kundin 2 Ankunft in Warteschlange
33.78762702788011 Kundin 2 Kassiervorgang beginnt (Wartezeit 0.0)
39.79189130033227 Kundin 1 Kassiervorgang endet
61.42857394352097 Kundin 2 Kassiervorgang endet
70.28967813149463 Kundin 3 Ankunft in Warteschlange
70.28967813149463 Kundin 3 Kassiervorgang beginnt (Wartezeit 0.0)
112.83399815123262 Kundin 3 Kassiervorgang endet
117.17151388748732 Kundin 4 Ankunft in Warteschlange
117.17151388748732 Kundin 4 Kassiervorgang beginnt (Wartezeit 0.0)
138.20918041290068 Kundin 4 Kassiervorgang endet
156.26166907957054 Kundin 5 Ankunft in Warteschlange
156.26166907957054 Kundin 5 Kassiervorgang beginnt (Wartezeit 0.0)
172.0184047543345 Kundin 5 Kassiervorgang endet
230.06838823168664 Kundin 6 Ankunft in Warteschlange
230.06838823168664 Kundin 6 Kassiervorgang beginnt (Wartezeit 0.0)
243.35791324480505 Kundin 7 Ankunft in War